## Comparison to yfinance

In [1]:
import os

import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

from yahooquery import Ticker
import yfinance as yf

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,}'.format
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)

In [2]:
symbol = 'aapl'

YF_aapl = yf.Ticker('aapl')
YQ_aapl = Ticker('aapl')

### Financials

#### yfinance

In [5]:
%%time
attrs = ['cashflow', 'balance_sheet', 'financials']
yf_d = {}
for attr in attrs:
    df = getattr(YF_aapl, attr)
    yf_d[attr] = df.shape[0]

Wall time: 3.41 s


#### yahooquery

In [6]:
%%time
methods = ['cash_flow', 'balance_sheet', 'income_statement']
yq_d = {}
for method in methods:
    df = getattr(YQ_aapl, method)()
    df.set_index(['asOfDate', 'periodType'], inplace=True)
    df = df.T
    yq_d[method] = df.shape[0]

Wall time: 1.58 s


In [7]:
# Comparison of the number of items returned for each
cash_flow = yf_d.pop('cashflow')
income_statement = yf_d.pop('financials')
yf_d.update({'cash_flow': cash_flow, 'income_statement': income_statement})
pd.DataFrame({'yahooquery': yq_d, 'yfinance': yf_d})

,yahooquery,yfinance
cash_flow,26,19
balance_sheet,66,25
income_statement,39,22


Yahooquery retrieves more cash flow, balance sheet, and income statement data than yfinance and quicker!

### Historical Pricing

In [6]:
# Compare downloads for all companies within the S&P500
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
sp500 = tables[0]['Symbol'].tolist()
sp500 = [symbol.replace(".", "-") for symbol in sp500]

#### yfinance

In [7]:
%%time
yf_data = yf.download(sp500, period='ytd', interval='1d', group_by='ticker')

[*********************100%***********************]  505 of 505 completed
Wall time: 42.5 s


In [8]:
type(yf_data)
yf_data.shape
yf_data.head()

pandas.core.frame.DataFrame

(94, 3030)

PSX                                        \
                         Open               High                Low   
Date                                                                  
2020-01-02 112.08000183105469 113.18000030517578 111.37999725341797   
2020-01-03             111.25 112.41999816894531 108.19000244140625   
2020-01-06 108.23999786376953 108.44000244140625 107.20999908447266   
2020-01-07 107.12999725341797 108.22000122070312 106.73999786376953   
2020-01-08 108.31999969482422 108.44000244140625 103.80999755859375   

                                                                        VFC  \
                        Close          Adj Close   Volume              Open   
Date                                                                          
2020-01-02 112.19999694824219 109.68885040283203  1607900 99.66999816894531   
2020-01-03 108.48999786376953  106.0618896484375  2236900  98.9000015258789   
2020-01-06 107.80999755859375 105.39710998535156  3917100              99.0   
2020-01-07 108.13999938964844 105.71971893310547  2687900 99.16999816894531   
2020-01-08  104.0999984741211 101.77013397216797  3452600              98.0   

                                                                  \
                        High               Low             Close   
Date                                                               
2020-01-02            100.25  99.4800033569336 100.2300033569336   
2020-01-03 99.94000244140625 98.76000213623047  99.5999984741211   
2020-01-06 99.43000030517578  98.5999984741211 99.20999908447266   
2020-01-07 99.97000122070312 98.55999755859375 98.72000122070312   
2020-01-08 98.18000030517578 96.80999755859375 97.04000091552734   

                                                     IPG                     \
                   Adj Close   Volume               Open               High   
Date                                                                          
2020-01-02 99.54398345947266  2555100 23.270000457763672 23.360000610351562   
2020-01-03 98.91828918457031  1966200  22.40999984741211 22.780000686645508   
2020-01-06 98.53096008300781  1967600 22.579999923706055 22.950000762939453   
2020-01-07 98.04431915283203  1766100   22.8799991607666 23.020000457763672   
2020-01-08 96.37581634521484  2980000 23.020000457763672  23.15999984741211   

                                                                              \
                          Low              Close          Adj Close   Volume   
Date                                                                           
2020-01-02 22.389999389648438 22.709999084472656 22.444355010986328  4513100   
2020-01-03  22.40999984741211 22.729999542236328 22.464120864868164  2302000   
2020-01-06 22.520000457763672  22.93000030517578  22.66178321838379  3321100   
2020-01-07 22.760000228881836 22.979999542236328 22.711196899414062  2853400   
2020-01-08  22.81999969482422 23.040000915527344 22.770496368408203  4438200   

                         BIIB                                        \
                         Open               High                Low   
Date                                                                  
2020-01-02 299.05999755859375  299.0799865722656  292.2799987792969   
2020-01-03  290.7699890136719 292.67999267578125  290.2200012207031   
2020-01-06   290.239990234375 291.19000244140625 288.20001220703125   
2020-01-07 290.57000732421875  291.8399963378906   287.989990234375   
2020-01-08  288.8800048828125 294.19000244140625 287.80999755859375   

                                                                        MNST  \
                        Close          Adj Close   Volume               Open   
Date                                                                           
2020-01-02   294.239990234375   294.239990234375  1178300 63.959999084472656   
2020-01-03  290.8500061035156  290.8500061035156   700100 63.119998931884766   
2020-01-06 290.82000732421875 290.82

#### yahooquery

In [9]:
%%time
tickers = Ticker(sp500, asynchronous=True)
yq_data = tickers.history(period='ytd', interval='1d')

Wall time: 12.3 s


In [10]:
type(yq_data)
yq_data.shape
yq_data.head()

pandas.core.frame.DataFrame

(47263, 8)

high              close   volume  \
symbol date                                                                 
MMM    2020-01-02 14:30:00 180.00999450683594              180.0  3601700   
       2020-01-03 14:30:00 178.66000366210938  178.4499969482422  2466900   
       2020-01-06 14:30:00  178.7100067138672  178.6199951171875  1998000   
       2020-01-07 14:30:00 178.50999450683594 177.89999389648438  2173000   
       2020-01-08 14:30:00              181.5  180.6300048828125  2758300   

                                         open                low  \
symbol date                                                        
MMM    2020-01-02 14:30:00 177.67999267578125 177.13999938964844   
       2020-01-03 14:30:00 177.02000427246094  175.6300048828125   
       2020-01-06 14:30:00 177.14999389648438 176.35000610351562   
       2020-01-07 14:30:00 178.27999877929688 176.82000732421875   
       2020-01-08 14:30:00              178.0 177.64999389648438   

                                     adjclose  dividends  splits  
symbol date                                                       
MMM    2020-01-02 14:30:00 178.38461303710938        0.0     0.0  
       2020-01-03 14:30:00 176.84852600097656        0.0     0.0  
       2020-01-06 14:30:00 177.01699829101562        0.0     0.0  
       2020-01-07 14:30:00 176.30345153808594        0.0     0.0  
       2020-01-08 14:30:00  179.0089569091797        0.0     0.0

### Option Chain

#### yfinance

In [8]:
%%time
dataframes = []
for option in YF_aapl.options:
    options = YF_aapl.option_chain(option)
    calls = options[0]
    calls['optionType'] = 'calls'
    puts = options[1]
    puts['optionType'] = 'puts'
    dataframes.append(pd.concat([calls, puts]))
yf_option_chain = pd.concat(dataframes)

Wall time: 4.11 s


In [9]:
yf_option_chain

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType
0,AAPL200807C00180000,2020-08-04 17:50:02,180.0,259.1,257.6,260.8,-4.669983,-1.7704755,3.0,3.0,4.4570356787109375,True,REGULAR,USD,calls
1,AAPL200807C00185000,2020-07-31 19:49:48,185.0,239.6,252.6,255.8,0.0,0.0,15.0,15.0,4.330082712402343,True,REGULAR,USD,calls
2,AAPL200807C00190000,2020-07-31 19:45:00,190.0,233.35,247.6,250.55,0.0,0.0,13.0,14.0,4.058598676757812,True,REGULAR,USD,calls
3,AAPL200807C00205000,2020-08-03 14:11:35,205.0,238.58,232.6,235.8,0.0,0.0,1.0,0.0,3.853515991210937,True,REGULAR,USD,calls
4,AAPL200807C00210000,2020-08-03 14:11:35,210.0,233.6,228.2,230.7,0.0,0.0,1.0,0.0,3.957031357421875,True,REGULAR,USD,calls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,AAPL220916P00500000,2020-08-03 16:44:25,500.0,119.34,119.05,124.0,0.0,0.0,16.0,72.0,0.33441064765930173,True,REGULAR,USD,puts
42,AAPL220916P00520000,2020-07-31 16:04:22,520.0,148.0,133.0,138.0,0.0,0.0,1.0,22.0,0.3336706037902832,True,REGULAR,USD,puts
43,AAPL220916P00540000,2020-07-28 18:12:36,540.0,186.0,147.5,152.5,0.0,0.0,40.0,45.0,0.3327856028747559,True,REGULAR,USD,puts
44,AAPL220916P00560000,2020-08-03 18:51:54,560.0,163.7,162.0,167.0,0.0,0.0,2.0,11.0,0.3298940912628173,True,REGULAR,USD,puts


#### yahooquery

In [10]:
%%time
yq_option_chain = YQ_aapl.option_chain

Wall time: 1.09 s


In [11]:
yq_option_chain

contractSymbol  strike currency  lastPrice  \
symbol expiration optionType                                                    
aapl   2020-08-07 calls       AAPL200807C00180000   180.0      USD      259.1   
                  calls       AAPL200807C00185000   185.0      USD      239.6   
                  calls       AAPL200807C00190000   190.0      USD     233.35   
                  calls       AAPL200807C00205000   205.0      USD     238.58   
                  calls       AAPL200807C00210000   210.0      USD      233.6   
...                                           ...     ...      ...        ...   
       2022-09-16 puts        AAPL220916P00500000   500.0      USD     119.34   
                  puts        AAPL220916P00520000   520.0      USD      148.0   
                  puts        AAPL220916P00540000   540.0      USD      186.0   
                  puts        AAPL220916P00560000   560.0      USD      163.7   
                  puts        AAPL220916P00600000   600.0      USD      197.0   

                                change  percentChange  volume  openInterest  \
symbol expiration optionType                                                  
aapl   2020-08-07 calls      -4.669983     -1.7704755     3.0           3.0   
                  calls            0.0            0.0    15.0          15.0   
                  calls            0.0            0.0    13.0          14.0   
                  calls            0.0            0.0     1.0           0.0   
                  calls            0.0            0.0     1.0           0.0   
...                                ...            ...     ...           ...   
       2022-09-16 puts             0.0            0.0    16.0          72.0   
                  puts             0.0            0.0     1.0          22.0   
                  puts             0.0            0.0    40.0          45.0   
                  puts             0.0            0.0     2.0          11.0   
                  puts             2.0      1.0256411     2.0           4.0   

                                bid    ask contractSize       lastTradeDate  \
symbol expiration optionType                                                  
aapl   2020-08-07 calls       257.6  260.8      REGULAR 2020-08-04 17:50:02   
                  calls       252.6  255.8      REGULAR 2020-07-31 19:49:48   
                  calls       247.6 250.55      REGULAR 2020-07-31 19:45:00   
                  calls       232.6  235.8      REGULAR 2020-08-03 14:11:35   
                  calls       228.2  230.7      REGULAR 2020-08-03 14:11:35   
...                             ...    ...          ...                 ...   
       2022-09-16 puts       119.05  124.0      REGULAR 2020-08-03 16:44:25   
                  puts        133.0  138.0      REGULAR 2020-07-31 16:04:22   
                  puts        147.5  152.5      REGULAR 2020-07-28 18:12:36   
                  puts        162.0  167.0      REGULAR 2020-08-03 18:51:54   
                  puts        193.5  198.5      REGULAR 2020-08-04 19:33:53   

                               impliedVolatility  inTheMoney  
symbol expiration optionType                                  
aapl   2020-08-07 calls       4.4570356787109375        True  
                  calls        4.330082712402343        True  
                  calls        4.058598676757812        True  
                  calls        3.853515991210937        True  
                  calls        3.957031357421875        True  
...                                          ...         ...  
       2022-09-16 puts       0.33441064765930173        True  
                  puts        0.3336706037902832        True  
                  puts        0.3327856028747559        True  
                  puts        0.3298940912628173        True  
                  puts        0.3288183573913574        True  

[2317 rows x 14 columns]

Retrieving all option expiration dates is a simple one-liner in yahooquery (and faster)!

### Multiple Symbols

In [4]:
symbols = 'FB AAPL AMZN NFLX GOOG'

#### yfinance

In [18]:
%%time
attrs = [
    'info', 'financials', 'quarterly_financials', 'major_holders',
    'institutional_holders', 'balance_sheet', 'quarterly_balance_sheet',
    'cashflow', 'quarterly_cashflow', 'earnings', 'quarterly_earnings',
    'sustainability', 'recommendations', 'calendar'
]
yf_d = {}
tickers = yf.Tickers(symbols)
for symbol in symbols.split(" "):
    ticker = getattr(tickers.tickers, symbol)
    yf_d[symbol] = {}
    for attr in attrs:
        yf_d[symbol][attr] = getattr(ticker, attr)

Wall time: 15.5 s


In [19]:
yf_d.keys()
yf_d['FB'].keys()

dict_keys(['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG'])

dict_keys(['info', 'financials', 'quarterly_financials', 'major_holders', 'institutional_holders', 'balance_sheet', 'quarterly_balance_sheet', 'cashflow', 'quarterly_cashflow', 'earnings', 'quarterly_earnings', 'sustainability', 'recommendations', 'calendar'])

#### yahooquery

In [5]:
%%time
tickers = Ticker(symbols, asynchronous=True)
data = tickers.all_modules

Wall time: 15.7 s


In [8]:
data['AAPL']['assetProfile']

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408-996-1010',
 'website': 'http://www.apple.com',
 'industry': 'Consumer Electronics',
 'sector': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides digital content stores and streaming services; AppleCare support services; and iCloud, a cloud service, which stores music, photos, contacts, calendars, mail, documents, and others. In addition, the company offers various service, such as Apple Arcade, a 

Again, retrieving data, this time with multiple symbols, is faster and simpler!